# Задание 10

### 10. Случайная величина имеет экспоненциальный закон распределения

При снятии показаний измерительного прибора десятые доли деления шкалы прибора оцениваются "на глаз" наблюдателем. Количества цифр 0, 1, 2, ..., 9, записанных наблюдателем в качестве десятых долей при 100 независимых измерениях, равны 5, 8, 6, 12, 14, 18, 11, 6, 13, 7 соответственно

#### a)
Проверить гипотезу о согласии данных с законом равномерного распределения с помощью критерия $\chi ^ 2$ и с помощью критерия Колмогорова. Сравнить результаты.

$H_0:$ Цифры равновероятны

$H_1: \overline H_0 $

##### Критерий Колмогорова

$$
\tilde{\Delta} = \sqrt{N} * \max_{x \in \mathbb{R}}{|\tilde {F}(x) - F(x)|}
$$

In [2]:
import numpy as np

N = 100

p = np.array([5,8,6,12,14,18,11,6,13,7])
q = np.array([1 for _ in range(10)])

F_emp = np.array([sum(p[:i]) for i in range(len(p)+1)]) / 100
F_th = np.array([sum(q[:i]) for i in range(len(q))]) / 10


delta = np.max([np.sqrt(N) * max(np.abs(F_th[i] - F_emp[i]),np.abs(F_th[i] - F_emp[i+1])) for i in range(F_th.size)])

delta

1.4000000000000001

$$
p-value = P(\Delta \geq \tilde{\Delta} | H_0)  = 1 - K(\tilde{\Delta}), где
$$

$$
K(x) = 1 + 2 * \sum_{k=1}^{\infty}(-1)^k \cdot e ^ {-2k^2 x^2}
$$

In [3]:
def K(x):
    res=1.0
    for i in range(1, 1001):
        res += 2 * (-1) ** i * np.exp(-2*(i**2)*(x**2))
    return res

p_val = 1-K(delta)
p_val

0.03968187953811442

Отвергаем гипотезу $H_0$, так как $p-value \le 0.05$

Критерий $\chi^2$ не отвергает гипотезу равномерного распределения, но критерий Колмогорова отвергает её. \
Критерий Колмогорова обнаружил систематические отклонения в распределении.

#### b)

Проверить гипотезу о согласии данных с законом нормального распределения с помощью критерия $\chi ^ 2$ и с помощью критерия Колмогорова. Сравнить результаты.

$H_0:$ Данные согласуются с нормальным распределением ~$N(\alpha, \sigma^2)$

$H_1: \overline H_0 $

In [5]:
import math

n_iterations = 50000

bootstrap_delta  = []

mean = 4.77
sigma = np.sqrt(6.34)

def n(x, mean_val, std_dev):
    return 0.5 * (1 + math.erf((x - mean_val)/(np.sqrt(2) * std_dev)))

x = [i for i in range(10)]
delta_wave = np.max([np.sqrt(N) * max(np.abs(n(x[i], mean, sigma) - F_emp[i]),np.abs(n(x[i], mean, sigma) - F_emp[i+1])) for i in range(10)])

for i in range(n_iterations):
    sample = np.random.normal(mean, sigma, N)
    sample_sorted = sorted(sample)
    bootstrap_F_emp = [i / N for i in range(N + 1)]
    bootstrap_mean = np.mean(sample)
    bootstrap_std_dev = np.std(sample) * N /(N - 1)
    bootstrap_delta.append(np.max([np.sqrt(N) * max(np.abs(n(sample_sorted[j], bootstrap_mean, bootstrap_std_dev) - bootstrap_F_emp[j]), \
                                     np.abs(n(sample_sorted[j], bootstrap_mean, bootstrap_std_dev) - bootstrap_F_emp[j+1])) \
                                        for j in range(len(sample_sorted))]))

bootstrap_delta = np.array(bootstrap_delta)
p_val = len(bootstrap_delta[bootstrap_delta >= delta_wave]) / n_iterations
p_val

0.01618

Отвергаем гипотезу $H_0$, так как $p-value \le 0.05$

Оба критерия отвергли гипотезу нормального распределения.
Критерий Коломогорова отверг гипотезу с меньшей мерой значимости.